## Analyzing World Factbook by CIA

The [World Factbook]() provides information on the history, people and society, government, economy, energy, geography, communications, transportation, military, and transnational issues for 267 world entities. Some of the demographic fields of the database are:
* population - The population as of 2015.
* population_growth - The annual population growth rate, as a percentage.
* area - The total land and water area.

Aim of the analysis is to answer various demographical questions about the nations in the database.
1. Which countries are densely populated ?
2. What country has the highest growth rate?
3. Which countries will add the most people to their population next year?
etc.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [6]:
%%sql 
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [7]:
%%sql
SELECT * FROM facts LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [8]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth) FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


A few things stick out from the summary statistics in the last screen:
1. There's a country with a population of 0
2. There's a country with a population of 7256490011 (or more than 7.2 billion people)

In [12]:
%%sql
SELECT name FROM facts WHERE population=(SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


name
Antarctica


In [11]:
%%sql
SELECT name FROM facts WHERE population=(SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


name
World


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

Recompute the summary statistics we found earlier while excluding the row for the whole world.

In [13]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth) FROM facts 
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [14]:
%%sql
SELECT AVG(population), AVG(area) FROM facts WHERE name != 'World'

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


#### Which countries have above average population and below average area ?

In [36]:
%%sql
SELECT name FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name NOT IN ('World', 'European Union')) AND 
        area < (SELECT AVG(area) FROM facts WHERE name NOT IN ('World', 'European Union'));

 * sqlite:///factbook.db
Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Malaysia
Morocco
Nepal
Philippines


#### Which countries have the most people ?

In [19]:
%%sql
SELECT name FROM facts WHERE name != 'World' ORDER BY population DESC LIMIT 10

 * sqlite:///factbook.db
Done.


name
China
India
European Union
United States
Indonesia
Brazil
Pakistan
Nigeria
Bangladesh
Russia


The database combines the european countries into a single entity called 'European Union', so we exclude 'European Union' and 'World' both query below.

In [22]:
%%sql
SELECT name, population FROM facts WHERE name NOT In ('World', 'European Union') ORDER BY population DESC LIMIT 10

 * sqlite:///factbook.db
Done.


name,population
China,1367485388
India,1251695584
United States,321368864
Indonesia,255993674
Brazil,204259812
Pakistan,199085847
Nigeria,181562056
Bangladesh,168957745
Russia,142423773
Japan,126919659


We can see the sudden jump from the third place United States at 321 million to India on second place with 1 billion two hundred fifty one million population.

#### Which countries have the highest growth rate?

In [25]:
%%sql
SELECT name,population_growth FROM facts WHERE name NOT IN ('World', 'European Union') ORDER BY population_growth DESC LIMIT 5

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24


#### Which countries have the highest ratios of water to land? 

In [27]:
%%sql
SELECT name, (CAST(area_water AS FLOAT)/area_land) AS ratio FROM facts 
WHERE name NOT IN ('World', 'European Union')
ORDER BY ratio DESC LIMIT 10;

 * sqlite:///factbook.db
Done.


name,ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


#### Which countries have more water than land?


In [29]:
%%sql
SELECT name FROM facts 
WHERE name NOT IN ('World', 'European Union') AND (area_water > area_land);

 * sqlite:///factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


#### Which countries will add the most people to their population next year?

In [31]:
%%sql
SELECT name, (population * population_growth/100) AS growth FROM facts
WHERE name NOT IN ('World', 'European Union')
ORDER BY growth DESC LIMIT 5

 * sqlite:///factbook.db
Done.


name,growth
India,15270686.1248
China,6153684.246
Nigeria,4448270.372
Pakistan,2906653.3662
Ethiopia,2874562.1691


#### Which countries have a higher death rate than birth rate?

In [30]:
%%sql
SELECT name FROM facts 
WHERE name NOT IN ('World', 'European Union') AND (death_rate > birth_rate);

 * sqlite:///factbook.db
Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary


#### What countries have the highest population/area ratio and how does it compare to list we found in the previous screen?

In [33]:
%%sql
SELECT name, (population/area) as density FROM facts
WHERE name NOT IN ('World', 'European Union')
ORDER BY density DESC 
LIMIT 5

 * sqlite:///factbook.db
Done.


name,density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
